In [10]:
import pandas as pd
from tqdm.notebook import tqdm
import re
from django.db.models import Q
from helpers.strings import get_obj_from_path

In [11]:
def traduz_operational_control(dicio):
    possible_lane_path = (
            "operational_control__fields__kind__selectoptions__options"
        )
    operational_control = get_obj_from_path(
            Company.objects.get(name=dicio['company__name']).custom_options, possible_lane_path
        )
    valor = next(a['name'] for a in operational_control if a['value'] == dicio['operational_control__kind'])
    dicio['opc__name'] = valor
    return dicio

In [2]:
filename = 'Dados Engie_v5'
# pd.DataFrame().to_excel(filename+'.xlsx')

In [142]:
ocrs=OccurrenceRecord.objects.all().values('company__name','datetime__month','datetime__year').annotate(qtd_apt=Count('uuid')).order_by('company__name')

In [143]:
ocrs

<QuerySet [{'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 1, 'datetime__year': 2019, 'qtd_apt': 12}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 1, 'datetime__year': 2020, 'qtd_apt': 9}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 2, 'datetime__year': 2019, 'qtd_apt': 7}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 2, 'datetime__year': 2020, 'qtd_apt': 1}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 3, 'datetime__year': 2019, 'qtd_apt': 2}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 3, 'datetime__year': 2020, 'qtd_apt': 8}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 4, 'datetime__year': 2019, 'qtd_apt': 11}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 4, 'datetime__year': 2020, 'qtd_apt': 6}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 5, 'datetime__year': 2019, 'qtd_apt': 5}, {'company__name': 'ATIVO-TREINAMENTO', 'datetime__month': 5, 'datetime__year': 2020, 'q

In [144]:
df = pd.DataFrame(data=ocrs,columns=['company__name','datetime__month','datetime__year','qtd_apt'])
df.columns=['Usina','Mês','Ano','Quantidade de Registros']
df

,Usina,Mês,Ano,Quantidade de Registros
0,ATIVO-TREINAMENTO,1.0,2019.0,12
1,ATIVO-TREINAMENTO,1.0,2020.0,9
2,ATIVO-TREINAMENTO,2.0,2019.0,7
3,ATIVO-TREINAMENTO,2.0,2020.0,1
4,ATIVO-TREINAMENTO,3.0,2019.0,2
...,...,...,...,...
980,Limbo - UTPS,8.0,2022.0,37
981,Limbo - UTPS,9.0,2022.0,53
982,POC-Projeto APR,5.0,2020.0,6
983,POC-Projeto APR,6.0,2020.0,1


In [145]:
df['Usina_inicio'] = df['Usina'].str.slice(stop=3)
dfSomaMesUsina = df.groupby(["Ano",'Mês','Usina_inicio'])['Quantidade de Registros'].sum()
df,dfSomaMesUsina

(                 Usina  Mês     Ano  Quantidade de Registros Usina_inicio
 0    ATIVO-TREINAMENTO  1.0  2019.0                       12          ATI
 1    ATIVO-TREINAMENTO  1.0  2020.0                        9          ATI
 2    ATIVO-TREINAMENTO  2.0  2019.0                        7          ATI
 3    ATIVO-TREINAMENTO  2.0  2020.0                        1          ATI
 4    ATIVO-TREINAMENTO  3.0  2019.0                        2          ATI
 ..                 ...  ...     ...                      ...          ...
 980       Limbo - UTPS  8.0  2022.0                       37          Lim
 981       Limbo - UTPS  9.0  2022.0                       53          Lim
 982    POC-Projeto APR  5.0  2020.0                        6          POC
 983    POC-Projeto APR  6.0  2020.0                        1          POC
 984  RECURSOS HIDRICOS  1.0  2023.0                       13          REC
 
 [985 rows x 5 columns],
 Ano     Mês   Usina_inicio
 1977.0  11.0  GSB                16
        

In [146]:
df_soma = df.groupby('Usina_inicio')['Quantidade de Registros'].sum()
df_soma

Usina_inicio
ATI       166
GES     42445
GSB    379736
GSO     11742
Lim        90
POC         7
REC        13
Name: Quantidade de Registros, dtype: int64

In [147]:
df2 = pd.DataFrame(data=df_soma)
df_final_qtd_registros = df2.transpose()[['GES','GSB','GSO','REC']]
df_final_qtd_registros = df_final_qtd_registros.transpose()
df_final_qtd_registros

,Quantidade de Registros
Usina_inicio,
GES,42445
GSB,379736
GSO,11742
REC,13


In [148]:
df_mes_soma = pd.DataFrame(data=dfSomaMesUsina)
df_mes_soma = df_mes_soma.reset_index()
df_mes_soma_v2 = df_mes_soma[df_mes_soma['Usina_inicio'].isin(['GES','GSB','GSO','REC'])].set_index(['Ano','Usina_inicio'])
df_mes_soma_v2

Mês  Quantidade de Registros
Ano    Usina_inicio                               
1977.0 GSB           11.0                       16
       GSB           12.0                       11
1978.0 GSB            1.0                       30
       GSB            2.0                       26
       GSB            3.0                       34
...                   ...                      ...
2023.0 GSB           11.0                    23225
       GES           12.0                      355
       GSB           12.0                    22733
2024.0 GES            1.0                       57
       GSB            1.0                    13016

[432 rows x 2 columns]

In [150]:
aclog=ActionLog.objects.filter(
    Q(company__name__startswith='GES')|Q(company__name__startswith='GSB')|Q(company__name__startswith='GSO')|Q(company__name__startswith='REC')
).values_list('company__name','created_at__month','created_at__year','action').annotate(qtd_atividade=Count('uuid')).order_by('company__name')

In [151]:
ActionLog._meta.fields,aclog

((<django.db.models.fields.UUIDField: uuid>,
  <django.db.models.fields.related.ForeignKey: company>,
  <django.db.models.fields.related.ForeignKey: company_group>,
  <django.db.models.fields.related.ForeignKey: user>,
  <django.db.models.fields.DateTimeField: created_at>,
  <django.db.models.fields.TextField: action>,
  <django.db.models.fields.TextField: user_ip>,
  <django.db.models.fields.TextField: user_port>,
  <django.db.models.fields.TextField: user_agent>,
  <django.db.models.fields.related.ForeignKey: content_type>,
  <django.db.models.fields.UUIDField: object_id>),
 <QuerySet [('GES - GEH', 1, 2021, 'Create', 1), ('GES - GEH', 1, 2022, 'Update', 1), ('GES - GEH', 1, 2023, 'Create', 2), ('GES - GEH', 1, 2023, 'Update', 5), ('GES - GEH', 1, 2024, 'Create', 13), ('GES - GEH', 1, 2024, 'Update', 25), ('GES - GEH', 2, 2023, 'Create', 66), ('GES - GEH', 2, 2023, 'Update', 57), ('GES - GEH', 3, 2022, 'Create', 41), ('GES - GEH', 3, 2022, 'Delete', 5), ('GES - GEH', 3, 2022, 'Update

In [152]:
df_action=pd.DataFrame(data=aclog)
df_action.rename(columns={0: 'Usina',1:'Mês',2:'Ano',3: 'Ação',4: 'Qtd por Ação'}, inplace=True)
df_action['Ação']=df_action['Ação'].str.replace('Create','Criação')
df_action['Ação']=df_action['Ação'].str.replace('Delete','Deleção')
df_action['Ação']=df_action['Ação'].str.replace('Update','Atualização')
df_action

,Usina,Mês,Ano,Ação,Qtd por Ação
0,GES - GEH,1,2021,Criação,1
1,GES - GEH,1,2022,Atualização,1
2,GES - GEH,1,2023,Criação,2
3,GES - GEH,1,2023,Atualização,5
4,GES - GEH,1,2024,Criação,13
...,...,...,...,...,...
1405,GSO - UTPS,7,2023,Atualização,1
1406,GSO - UTPS,8,2023,Atualização,2
1407,GSO - UTPS,10,2023,Atualização,3
1408,RECURSOS HIDRICOS,1,2024,Atualização,1


In [153]:
df_action['Usina_inicio'] = df_action['Usina'].str.slice(stop=3)
df_action_soma = df_action.groupby('Usina_inicio')['Qtd por Ação'].sum()
df_action_soma_anomes = df_action.groupby(['Ano','Mês','Usina_inicio'])['Qtd por Ação'].sum()
df_action_soma_anomes

Ano   Mês  Usina_inicio
2020  8    GES              969
      9    GES             3181
      10   GES             3066
      11   GES             2402
           GSO                4
                           ... 
2023  12   GSB              118
           GSO               14
2024  1    GES             3203
           GSO                3
           REC                1
Name: Qtd por Ação, Length: 72, dtype: int64

In [154]:
df2_action_soma_anomes = pd.DataFrame(data=df_action_soma_anomes)
df2_action_soma_anomes

Qtd por Ação
Ano  Mês Usina_inicio              
2020 8   GES                    969
     9   GES                   3181
     10  GES                   3066
     11  GES                   2402
         GSO                      4
...                             ...
2023 12  GSB                    118
         GSO                     14
2024 1   GES                   3203
         GSO                      3
         REC                      1

[72 rows x 1 columns]

In [155]:
df2_action_soma = pd.DataFrame(data=df_action_soma)
df_final_action_soma = df2_action_soma.transpose()[['GES','GSB','GSO','REC']]
df2_action_soma.rename(columns={'Qtd por Ação':'Total de ações no sistema por usina'},inplace=True)
df2_action_soma

,Total de ações no sistema por usina
Usina_inicio,
GES,173062
GSB,988
GSO,1085
REC,2


In [156]:
occs = OccurrenceType.objects.all().values('company__name').annotate(qtd_forms=Count('uuid')).distinct()
df_forms=pd.DataFrame(data=occs)
df_forms['Usina_inicio'] = df_forms['company__name'].str.slice(stop=3)
df_forms_soma = df_forms.groupby('Usina_inicio')['qtd_forms'].sum()
df2_forms_soma = pd.DataFrame(data=df_forms_soma)
df2_final_forms_soma = df2_forms_soma.transpose()[['GES','GSB','GSO','REC']]
df2_final_forms_soma=df2_final_forms_soma.transpose()
df2_final_forms_soma.rename(columns={'qtd_forms':'Quantidade de Formularios por Usina'},inplace=True)
df2_final_forms_soma

,Quantidade de Formularios por Usina
Usina_inicio,
GES,733
GSB,499
GSO,125
REC,2


In [157]:
distinct_classes={'GES':OccurrenceType.objects.filter(company__name__startswith='GES').distinct().count(),
'GSB':OccurrenceType.objects.filter(company__name__startswith='GSB').distinct().count(),
'GSO':OccurrenceType.objects.filter(company__name__startswith='GSO').distinct().count(),
'REC':OccurrenceType.objects.filter(company__name__startswith='REC').distinct().count()}
distinct_classes

{'GES': 109, 'GSB': 164, 'GSO': 9, 'REC': 2}

In [158]:
for k,v in distinct_classes.items():
    df2_final_forms_soma.loc[k] = v
df2_final_forms_soma

,Quantidade de Formularios por Usina
Usina_inicio,
GES,109
GSB,164
GSO,9
REC,2


In [162]:
OccurrenceRecord.objects.filter(
    company__name__startswith='GES',
    datetime__year='2023'
).exclude(
    created_by__username='engie'
).count()

10603

In [171]:
occ_used=OccurrenceRecord.objects.filter(
    company__name__startswith='GES'
).values(
    'occurrence_type__name','occurrence_type__occurrence_kind'
).annotate(
    qtd_por_classe=Count('uuid')
).order_by(
    '-qtd_por_classe'
)

In [172]:
occ_used[:5]

<QuerySet [{'occurrence_type__name': 'Geração do resíduo', 'occurrence_type__occurrence_kind': '704', 'qtd_por_classe': 10766}, {'occurrence_type__name': 'Cadastro', 'occurrence_type__occurrence_kind': '709', 'qtd_por_classe': 9303}, {'occurrence_type__name': 'Resumo Executivo', 'occurrence_type__occurrence_kind': '4', 'qtd_por_classe': 3916}, {'occurrence_type__name': 'Ocorrências patrimoniais', 'occurrence_type__occurrence_kind': '1', 'qtd_por_classe': 2762}, {'occurrence_type__name': 'Vistorias técnicas', 'occurrence_type__occurrence_kind': '5', 'qtd_por_classe': 2735}]>

In [164]:
total = sum([item['qtd_por_classe'] for item in list(occ_used)])
total

42445

In [165]:
percentage=list(map(lambda x: x/total, [item['qtd_por_classe'] for item in list(occ_used[:5])]))

In [166]:
temp=[]
for a in zip(list(occ_used[:5]),percentage):
    temp.append({'Classe':a[0]['occurrence_type__name'],
                 'Top 5 Mais usados':a[0]['qtd_por_classe'],
                 'Porcentagem':f'{round(a[1]*100,2)}%'})

In [167]:
temp

[{'Classe': 'Geração do resíduo',
  'Top 5 Mais usados': 10766,
  'Porcentagem': '25.36%'},
 {'Classe': 'Cadastro', 'Top 5 Mais usados': 10757, 'Porcentagem': '25.34%'},
 {'Classe': 'Resumo Executivo',
  'Top 5 Mais usados': 3916,
  'Porcentagem': '9.23%'},
 {'Classe': 'Ocorrências patrimoniais',
  'Top 5 Mais usados': 2762,
  'Porcentagem': '6.51%'},
 {'Classe': 'Vistorias técnicas',
  'Top 5 Mais usados': 2735,
  'Porcentagem': '6.44%'}]

In [168]:
df_top_5=pd.DataFrame(data=temp)

In [169]:
df_top_5.set_index('Classe',inplace=True)
df_top_5

,Top 5 Mais usados,Porcentagem
Classe,,
Geração do resíduo,10766,25.36%
Cadastro,10757,25.34%
Resumo Executivo,3916,9.23%
Ocorrências patrimoniais,2762,6.51%
Vistorias técnicas,2735,6.44%


In [170]:
with pd.ExcelWriter(filename+'.xlsx', mode='w',engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name='Qtd Registro por Usina', index=False)
    df_final_qtd_registros.to_excel(writer, sheet_name='Qtd Reg por Subsis', index=True)
    df_mes_soma_v2.to_excel(writer, sheet_name='Qtd-Reg-Ano-Mes-Subsis', index=True)
    df_action.to_excel(writer, sheet_name='Atividades por Usina', index=False)
    df2_action_soma.to_excel(writer, sheet_name='Total Ações por Subsis', index=True)
    df2_action_soma_anomes.to_excel(writer, sheet_name='Total-Ações-Ano-Mes-Subsis', index=True)
    df2_final_forms_soma.to_excel(writer, sheet_name='Total Form por Subsis', index=True)
    df_top_5.to_excel(writer, sheet_name='Top 5 classes mais usadas', index=True)

In [ ]:
print('Registro em 2022:',OccurrenceRecord.objects.filter(
    company__name__startswith='GSO',
    datetime__year='2022'
).count())
print('Registro em 2023:',OccurrenceRecord.objects.filter(
    company__name__startswith='GSO',
    datetime__year='2023'
).count())

In [ ]:
OccurrenceRecord.objects.filter(
    company__name__startswith='GSO',
    datetime__year='2023'
)[0]

# Outro relatórios de usuários
-Número de usuários por unidade do sistema (ENGIE)
-Número de usuários únicos por sistema (GES, GSO, GSB e GRH)
-Número de registro por controle operacional e por usina
--Ex: Educação ambiental em Itá = 1500; Educação ambiental em Machadinho = 200 e assim por diante para cada controle operacional --configurado
-Número de usuários com pelo menos 1 ação no sistema no último ano (2023) (independente de unidade)
-Número total de usuários cadastrados no sistema
-Número total de usuários cadastrados na plataforma principal da Kartado (não no ambiente da ENGIE). Nesse indicador precisaria --somar usuários cadastrados no ambiente compartilhado + CCR

In [31]:
uic = UserInCompany.objects.all()
company=Company.objects.filter(Q(name__startswith='REC') | Q(name__startswith='G'))

In [32]:
# Usuários continuam saindo duplicados foi utilizado outra forma de trazer os dados. 
# Esse script para trazer esses dados esta descontinuado.
dup=[]
dict_distinct=[]
for c in company:
    total=0
    for user in uic.filter(company=c):
        if str(user.uuid) not in dup:
            total+=1
            dup.append(str(user.uuid))
    dict_distinct.append({'Unidade':c.name,'Qtd Usuarios por Subsistema': total})

Exception ignored in: <function ZipFile.__del__ at 0x7f2d263823b0>
Traceback (most recent call last):
  File "/home/luan/.pyenv/versions/3.7.9/lib/python3.7/zipfile.py", line 1809, in __del__
    self.close()
  File "/home/luan/.pyenv/versions/3.7.9/lib/python3.7/zipfile.py", line 1826, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [33]:
df_users = pd.DataFrame(data=dict_distinct,columns=['Unidade','Qtd Usuarios por Subsistema'])
df_users

,Unidade,Qtd Usuarios por Subsistema
0,GSO - UHE-CANA BRAVA,29
1,GSO - UHE-SALTO OSORIO,23
2,GSO - UTPS,14
3,GSO - UHE-SALTO SANTIAGO,23
4,GSB - UHE-MACHADINHO,25
5,GSB - UHE-SALTO OSORIO,23
6,GSO - UHE-PONTE DE PEDRA,23
7,GSB - UHE-SALTO SANTIAGO,32
8,GSO - UHE-JAGUARA,24
9,GSO - UHE-MIRANDA,23


In [34]:
df_users['Usina_inicio'] = df_users['Unidade'].str.slice(stop=3)
dfSomaRegUsina = df_users.groupby(['Usina_inicio'])['Qtd Usuarios por Subsistema'].sum()
df_users,dfSomaRegUsina

(                            Unidade  Qtd Usuarios por Subsistema Usina_inicio
 0              GSO - UHE-CANA BRAVA                           29          GSO
 1            GSO - UHE-SALTO OSORIO                           23          GSO
 2                        GSO - UTPS                           14          GSO
 3          GSO - UHE-SALTO SANTIAGO                           23          GSO
 4              GSB - UHE-MACHADINHO                           25          GSB
 5            GSB - UHE-SALTO OSORIO                           23          GSB
 6          GSO - UHE-PONTE DE PEDRA                           23          GSO
 7          GSB - UHE-SALTO SANTIAGO                           32          GSB
 8                 GSO - UHE-JAGUARA                           24          GSO
 9                 GSO - UHE-MIRANDA                           23          GSO
 10  GSO - PCH-JOSÉ GELÁZIO DA ROCHA                           23          GSO
 11           GSO - PCH-RONDONÓPOLIS                

In [35]:
df_soma_soma = pd.DataFrame(data=dfSomaRegUsina)
df_soma_soma = df_soma_soma.reset_index()
df_soma_soma_v2 = df_soma_soma[df_soma_soma['Usina_inicio'].isin(['GES','GSB','GSO','REC'])].set_index(['Usina_inicio'])
df_soma_soma_v2

,Qtd Usuarios por Subsistema
Usina_inicio,
GES,1531
GSB,267
GSO,386
REC,1


In [36]:
ocrs_oper_con=OccurrenceRecord.objects.filter(
    Q(company__name__startswith='REC') |
    Q(company__name__startswith='G'),
operational_control__isnull=False).values('company__name','operational_control__kind').annotate(qtd_reg_opcon=Count('uuid'))

In [37]:
ocrs_oper_con[0],len(ocrs_oper_con)

({'company__name': 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
  'operational_control__kind': '704',
  'qtd_reg_opcon': 51},
 98)

In [38]:
traduz_operational_control(ocrs_oper_con[0])

{'company__name': 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
 'operational_control__kind': '704',
 'qtd_reg_opcon': 51,
 'opc__name': 'Resíduos'}

In [39]:
new_oper=[]
for a in tqdm(ocrs_oper_con):
    try:
        traduz_operational_control(a)
    except:
        a['opc__name'] = 'Auscultação - Vertedouro 1'

  0%|          | 0/98 [00:00<?, ?it/s]

In [40]:
for a in ocrs_oper_con:
    if '708' in a['operational_control__kind']:
        print(a)

{'company__name': 'GES - UHE-PONTE DE PEDRA', 'operational_control__kind': '708', 'qtd_reg_opcon': 11, 'opc__name': 'Salvamento de fauna'}
{'company__name': 'GSB - UHE-ITA', 'operational_control__kind': '708', 'qtd_reg_opcon': 216, 'opc__name': 'Auscultação - Vertedouro 1'}
{'company__name': 'GSB - UHE-SALTO OSORIO', 'operational_control__kind': '708', 'qtd_reg_opcon': 32, 'opc__name': 'Auscultação - Vertedouro 1'}
{'company__name': 'GSB - UHE-SALTO SANTIAGO', 'operational_control__kind': '708', 'qtd_reg_opcon': 323551, 'opc__name': 'Auscultação - Barragem Auxiliar 1 - Instrumentos automatizados'}


In [41]:
df_oper = pd.DataFrame(data=ocrs_oper_con,
                       columns=['company__name','operational_control__kind','qtd_reg_opcon','opc__name'])
df_oper.rename(columns={'company__name': 'Usina',
                        'operational_control__kind':'operational_control__kind',
                        'qtd_reg_opcon':'Quantidade de Registros','opc__name': 'Controle Operacional'}, inplace=True)
df_oper.drop(columns='operational_control__kind')
df_oper = df_oper.reindex(columns=['Usina', 'Controle Operacional','Quantidade de Registros'])
df_oper=df_oper.sort_values('Usina')
df_oper

,Usina,Controle Operacional,Quantidade de Registros
0,GES - PCH-JOSÉ GELÁZIO DA ROCHA,Resíduos,51
1,GES - PCH-RONDONÓPOLIS,Resíduos,51
2,GES - UHE-CANA BRAVA,Horto Florestal,4
3,GES - UHE-CANA BRAVA,Programa de visitas / Educação ambiental,3
4,GES - UHE-CANA BRAVA,Resíduos,246
...,...,...,...
92,GSB - UHE-SALTO SANTIAGO,Auscultação - Barragem Auxiliar 2,93
93,GSB - UHE-SALTO SANTIAGO,Auscultação - Vertedouro,3282
95,GSB - UHE-SALTO SANTIAGO,Auscultação - Barragem Auxiliar 2 - Instrument...,26105
96,GSB - UHE-SÃO SALVADOR,Auscultação - Instrumentos,241


In [195]:
df_mesclado = df_oper.groupby('Usina').apply(lambda x: x.reset_index(drop=True))
df_mesclado

Controle Operacional  \
Usina                                                                                  
GES - PCH-JOSÉ GELÁZIO DA ROCHA 0                                           Resíduos   
GES - PCH-RONDONÓPOLIS          0                                           Resíduos   
GES - UHE-CANA BRAVA            0                                    Horto Florestal   
                                1           Programa de visitas / Educação ambiental   
                                2                                           Resíduos   
...                                                                              ...   
GSB - UHE-SALTO SANTIAGO        6                  Auscultação - Barragem Auxiliar 2   
                                7                           Auscultação - Vertedouro   
                                8  Auscultação - Barragem Auxiliar 2 - Instrument...   
GSB - UHE-SÃO SALVADOR          0                         Auscultação - Instrumentos   
RECURSOS HIDRICOS               0  Pontos e parâmetros de monitoramento hidromete...   

                                   Quantidade de Registros  
Usina                                                       
GES - PCH-JOSÉ GELÁZIO DA ROCHA 0                       51  
GES - PCH-RONDONÓPOLIS          0                       51  
GES - UHE-CANA BRAVA            0                        4  
                                1                        3  
                                2                      246  
...                                                    ...  
GSB - UHE-SALTO SANTIAGO        6                       93  
                                7                     3282  
                                8                    26101  
GSB - UHE-SÃO SALVADOR          0                      241  
RECURSOS HIDRICOS               0                       13  

[98 rows x 2 columns]

In [208]:
df_oper

,Controle Operacional,Quantidade de Registros
Usina,,
GES - PCH-JOSÉ GELÁZIO DA ROCHA,Resíduos,51
GES - PCH-RONDONÓPOLIS,Resíduos,51
GES - UHE-CANA BRAVA,Horto Florestal,4
GES - UHE-CANA BRAVA,Programa de visitas / Educação ambiental,3
GES - UHE-CANA BRAVA,Resíduos,246
...,...,...
GSB - UHE-SALTO SANTIAGO,Auscultação - Barragem Auxiliar 2,93
GSB - UHE-SALTO SANTIAGO,Auscultação - Vertedouro,3282
GSB - UHE-SALTO SANTIAGO,Auscultação - Barragem Auxiliar 2 - Instrument...,26101


#### Número de usuários com pelo menos 1 ação no sistema no último ano (2023) (independente de unidade)

In [197]:
df_mesclado.index

MultiIndex([('GES - PCH-JOSÉ GELÁZIO DA ROCHA', 0),
            (         'GES - PCH-RONDONÓPOLIS', 0),
            (           'GES - UHE-CANA BRAVA', 0),
            (           'GES - UHE-CANA BRAVA', 1),
            (           'GES - UHE-CANA BRAVA', 2),
            (           'GES - UHE-CANA BRAVA', 3),
            (             'GES - UHE-ESTREITO', 0),
            (             'GES - UHE-ESTREITO', 1),
            (             'GES - UHE-ESTREITO', 2),
            (             'GES - UHE-ESTREITO', 3),
            (                  'GES - UHE-ITA', 0),
            (                  'GES - UHE-ITA', 1),
            (                  'GES - UHE-ITA', 2),
            (                  'GES - UHE-ITA', 3),
            (                  'GES - UHE-ITA', 4),
            (                  'GES - UHE-ITA', 5),
            (              'GES - UHE-JAGUARA', 0),
            (              'GES - UHE-JAGUARA', 1),
            (              'GES - UHE-JAGUARA', 2),
            

In [206]:
df_mesclado.set_index('Usina')

KeyError: "None of ['Usina'] are in the columns"

In [42]:
aclog_v2=ActionLog.objects.filter(
    Q(company__name__startswith='GES')|
    Q(company__name__startswith='GSB')|
    Q(company__name__startswith='GSO')|
    Q(company__name__startswith='REC'),
    created_at__year='2023'
).order_by(
    'user'
).distinct(
    'user'
).count()

In [207]:
df_mesclado.index(names=['Usina','level'])

TypeError: 'MultiIndex' object is not callable

In [43]:
df_aclog_v2 = pd.DataFrame(data={"Nº de user com pelo menos uma ação":aclog_v2},index=['Total de Usuários'])
df_aclog_v2

,Nº de user com pelo menos uma ação
Total de Usuários,113


#### Número total de usuários cadastrados no sistema Engie

In [44]:
all_user_engie = User.objects.all().count()
all_user_engie

767

#### Número total de usuários cadastrados na plataforma principal da Kartado (não no ambiente da ENGIE). Nesse indicador precisaria somar usuários cadastrados no ambiente compartilhado + CCR

In [45]:
all_user_ccr = User.objects.all().count()
all_user_ccr

767

In [46]:
df_all_user_infra = pd.DataFrame(data={"Ambiente Compartilhado":6284,'Engie':767,'CCR':448},index=['Total de Usuários'])
df_all_user_infra

,Ambiente Compartilhado,Engie,CCR
Total de Usuários,6284,767,448


In [8]:
distinct_uic={'GES':UserInCompany.objects.filter(company__name__startswith='GES').order_by('user').distinct('user').count(),
'GSB':UserInCompany.objects.filter(company__name__startswith='GSB').order_by('user').distinct('user').count(),
'GSO':UserInCompany.objects.filter(company__name__startswith='GSO').order_by('user').distinct('user').count(),
'REC':UserInCompany.objects.filter(company__name__startswith='REC').order_by('user').distinct('user').count()}
distinct_uic

{'GES': 563, 'GSB': 33, 'GSO': 82, 'REC': 1}

In [16]:
df2_final_uic = pd.DataFrame(columns=['Quantidade Usuários por Subsistema'])

In [17]:
for k,v in distinct_uic.items():
    df2_final_uic.loc[k] = v
df2_final_uic

,Quantidade Usuários por Subsistema
GES,563
GSB,33
GSO,82
REC,1


In [47]:
with pd.ExcelWriter('Rel_Users_Actions.xlsx', mode='w',engine="openpyxl") as writer:
#     df_users.to_excel(writer, sheet_name='Qtd Usuários por Usina', index=False)
    df2_final_uic.to_excel(writer, sheet_name='Qtd Usuarios por Subsis', index=True)
    df_oper.to_excel(writer, sheet_name='Qtd Reg-Controle-Oper', index=True)
    df_aclog_v2.to_excel(writer, sheet_name='User uma Acao', index=True)
    df_all_user_infra.to_excel(writer, sheet_name='Total User por ambiente', index=True)

In [7]:
UserInCompany.objects.filter(company__name__startswith='REC').order_by('user').distinct('user').count()

1